In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

import joblib

2024-02-26 05:57:57.250087: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 05:57:57.252860: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 05:57:57.284447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 05:57:57.284490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 05:57:57.286063: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_train.csv', low_memory=False)
x_val = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_val.csv', low_memory=False)
y_val = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_val.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_test.csv', low_memory=False)

In [4]:
from sklearn import preprocessing
s1 = preprocessing.MinMaxScaler()
s2 = preprocessing.MinMaxScaler()
s3 = preprocessing.MinMaxScaler()

x_train_scaled = s1.fit_transform(x_train)
x_val_scaled = s2.fit_transform(x_val)
x_test_scaled = s3.fit_transform(x_test)

x_train = np.copy(x_train_scaled)
x_val = np.copy(x_val_scaled)
x_test = np.copy(x_test_scaled)

In [5]:
model = joblib.load("/home/jovyan/WUSTL-IIoT/dl/dl_new/dnn.joblib")

In [6]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=5,
    input_shape=(48,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [7]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.01)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [8]:
calculate_performance_metrics(adv_test_examples, y_test, model)

7466/7466 [==============================] - 5s 587us/step
Accuracy: 0.9996316357882575
AUC: 0.9999587922999147

macro
Precision: 0.8546620972836386
Recall: 0.9230980771564614
F1 Score: 0.8506926062079415

weighted
Precision: 0.9998586600158942
Recall: 0.9996316357882575
F1 Score: 0.9997090276077586

Mean FNR: 0.07690192284353858
Mean TNR: 0.9999261362349952
Mean FPR: 7.386376500496653e-05
Mean TPR: 0.9230980771564614


In [9]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.05)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [10]:
calculate_performance_metrics(adv_test_examples, y_test, model)

7466/7466 [==============================] - 4s 564us/step
Accuracy: 0.9730005776620593
AUC: 0.9908790127108805

macro
Precision: 0.5989956927963804
Recall: 0.7132930863590052
F1 Score: 0.5309286280469487

weighted
Precision: 0.9886878900292506
Recall: 0.9730005776620593
F1 Score: 0.9777752018518082

Mean FNR: 0.2867069136409948
Mean TNR: 0.9675463675918925
Mean FPR: 0.0324536324081076
Mean TPR: 0.7132930863590052


In [7]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.1)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [8]:
adv_test_examples

array([[ 0.33368954,  0.3336235 ,  0.3846154 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.67549276,  0.67559546,  0.3846154 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.48704186,  0.48715675,  0.33076924, ..., -0.07680976,
         0.1       ,  0.9       ],
       ...,
       [ 0.20193145,  0.2019071 ,  0.3846154 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.2299533 ,  0.2301179 ,  0.30769232, ...,  0.01442168,
         0.10462401,  0.        ],
       [ 0.516726  ,  0.51679194,  0.61538464, ...,  0.        ,
         0.1       ,  0.9       ]], dtype=float32)

In [9]:
calculate_performance_metrics(adv_test_examples, y_test, model)

7466/7466 [==============================] - 4s 554us/step
Accuracy: 0.9294456955804666
AUC: 0.8752588989821266

macro
Precision: 0.5398146001350101
Recall: 0.43407474981921335
F1 Score: 0.358654722298251

weighted
Precision: 0.957817143438853
Recall: 0.9294456955804666
F1 Score: 0.9149545175216659

Mean FNR: 0.5659252501807868
Mean TNR: 0.896355697745137
Mean FPR: 0.10364430225486294
Mean TPR: 0.43407474981921335


In [10]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.15)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [11]:
adv_test_examples

array([[0.33368954, 0.3336235 , 0.3846154 , ..., 0.        , 0.        ,
        0.        ],
       [0.67549276, 0.67559546, 0.3846154 , ..., 0.        , 0.        ,
        0.        ],
       [0.5370419 , 0.53715676, 0.38076925, ..., 0.17319024, 0.15      ,
        0.85      ],
       ...,
       [0.20193145, 0.2019071 , 0.3846154 , ..., 0.        , 0.        ,
        0.        ],
       [0.2299533 , 0.2301179 , 0.30769232, ..., 0.01442168, 0.10462401,
        0.        ],
       [0.566726  , 0.56679195, 0.61538464, ..., 0.        , 0.15      ,
        0.85      ]], dtype=float32)

In [12]:
calculate_performance_metrics(adv_test_examples, y_test, model)

7466/7466 [==============================] - 4s 546us/step
Accuracy: 0.9278968915083677
AUC: 0.6429491885399685

macro
Precision: 0.5124657807786386
Recall: 0.33517829110028363
F1 Score: 0.3355800972634787

weighted
Precision: 0.9350213697128708
Recall: 0.9278968915083677
F1 Score: 0.9032354235500222

Mean FNR: 0.6648217088997164
Mean TNR: 0.8392157151721997
Mean FPR: 0.16078428482780016
Mean TPR: 0.33517829110028363


In [8]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.2)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [9]:
calculate_performance_metrics(adv_test_examples, y_test, model)

7466/7466 [==============================] - 4s 555us/step
Accuracy: 0.9277085234455449
AUC: 0.6046158278872422

macro
Precision: 0.5110308085957888
Recall: 0.32644378976501554
F1 Score: 0.33359085694279983

weighted
Precision: 0.9290163178191778
Recall: 0.9277085234455449
F1 Score: 0.8995574555667065

Mean FNR: 0.6735562102349844
Mean TNR: 0.8199846897217163
Mean FPR: 0.18001531027828369
Mean TPR: 0.32644378976501554


In [10]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.25)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [11]:
calculate_performance_metrics(adv_test_examples, y_test, model)

7466/7466 [==============================] - 4s 565us/step
Accuracy: 0.9276834077038352
AUC: 0.5703854682974475

macro
Precision: 0.5107003430231337
Recall: 0.3264383689518032
F1 Score: 0.33341217913432536

weighted
Precision: 0.9274852029030842
Recall: 0.9276834077038352
F1 Score: 0.8987296045578035

Mean FNR: 0.6735616310481968
Mean TNR: 0.8155593509569842
Mean FPR: 0.18444064904301588
Mean TPR: 0.3264383689518032


In [12]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.3)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [13]:
calculate_performance_metrics(adv_test_examples, y_test, model)

7466/7466 [==============================] - 4s 547us/step
Accuracy: 0.9276834077038352
AUC: 0.5698005009882172

macro
Precision: 0.5104870949372188
Recall: 0.3264383689518032
F1 Score: 0.33331066243223734

weighted
Precision: 0.926996040190814
Recall: 0.9276834077038352
F1 Score: 0.898468995872493

Mean FNR: 0.6735616310481968
Mean TNR: 0.8141528816090782
Mean FPR: 0.18584711839092188
Mean TPR: 0.3264383689518032
